In [2]:
# Constants
# declaring data path
RAW_DATA_PATH = "../data/raw/survey_results_public.csv"
CLEANED_DATA_PATH = "../data/processed/processed_df.pkl"
#  Columns of interest
SELECTED_COLUMNS =['DevType', 'EdLevel', 'YearsCode', 'YearsCodePro',
    'LanguageHaveWorkedWith', 'LanguageWantToWorkWith',
    'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith',
    'PlatformHaveWorkedWith', 'PlatformWantToWorkWith',
    'WebframeHaveWorkedWith', 'WebframeWantToWorkWith',
    'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith',
    'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith',
    'OpSysProfessional use', 'ProfessionalTech','NEWCollabToolsHaveWorkedWith'
]

In [3]:
# importing needed libraries
import pandas as pd
import numpy as np

In [4]:
raw_df= pd.read_csv(RAW_DATA_PATH)[SELECTED_COLUMNS]

In the previous notebook, we conclude the processing we need to do:

1- YearsCodePro , YearsCode : 
    * Data type converstion(from object -> int) 
    * convert ['Less than 1 year','More than 50 years'] to suitable value.
    * convert nan to suitable value.

2- DevType:
    * nan to suitable value.
    * Removing rows with values is in: ['Marketing or sales professional' ,'Designer', 'Student', 'Other (please specify):']
    
3- Split multiple answers that are seperated by ';' in skills and languages related columns.

#### Functions that we will need in the preprocessing:

In [5]:
def clean_date(df: pd.DataFrame, col: str) -> pd.Series:
    """
    This function cleans the specified column (`col`) in the given DataFrame (`df`).
    It replaces 'Less than 1 year' value with 0 and 'More than 50 years' with 51,
    converts the column from object to integer dtype, and replaces NaN values with the median of the column.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing the column to be cleaned.
    col (str): The name of the column to be cleaned.

    Returns:
    pd.Series: The cleaned column as a Series.
    """
    # Replace 'Less than 1 year' value by 0 and 'More than 50 years' by 51
    df[col] = df[col].replace(['Less than 1 year', 'More than 50 years'], [0, 51])

    # Convert column from object to float dtype
    df[col] = df[col].astype(float)    

    # Replace NaN values with median
    df[col] = df[col].fillna(df[col].median())

    return df[col]

def clean_target(df:pd.DataFrame,col:str)->pd.DataFrame:
    """
    This function removes rows from the given DataFrame (`df`) where the specified column (`col`) contains values in the following list:
    ['Marketing or sales professional' ,'Designer', 'Student', 'Other (please specify):',nan]

    Parameters:
    df (pd.DataFrame): The input DataFrame containing the column to be cleaned.
    col (str): The name of the column to be cleaned.

    Returns:
    pd.DataFrame: The DataFrame with rows containing the specified values removed.
    """
    # Remove rows with the specified values
    df = df[~df[col].isin(['Marketing or sales professional' ,'Designer', 'Student', 'Other (please specify):',np.nan])]

    return df
    
def split_multiple_answers(df:pd.DataFrame,splitter:str=';')->pd.DataFrame:
    """
    This function splits multiple answers that are separated by a specific delimiter in the given DataFrame.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing the columns to be split.
    splitter (str): The delimiter used to separate the values in the columns. Default is ';'

    Returns:
    pd.DataFrame: The DataFrame with the split columns.
    """

    def is_splittable_column(df:pd.DataFrame,col:str)-> bool:
        """
        This function checks if the specified column contains the splitter string value.

        Parameters:
        df (pd.DataFrame): The input DataFrame containing the column to be checked.
        col (str): The name of the column to be checked.

        Returns:
        bool: True if the column contains the splitter string value, False otherwise.
        """
        return df[col].str.contains(splitter).any()
        
    def split_values_in_column(df:pd.DataFrame,col:str)->pd.Series:
        """
        This function splits the values in the specified column (`col`) in the given DataFrame (`df`) using the splitter string value.

        Parameters:
        df (pd.DataFrame): The input DataFrame containing the column to be split.
        col (str): The name of the column to be split.
        splitter (str): The string value to be used as a separator when splitting the values in the column.

        Returns:
        pd.Series: The split column as a Series.
        """
        # Split the column values using the specified splitter    
        return df[col].str.split(splitter, expand=False)

    for col in df.select_dtypes(include=['object']):
        if(is_splittable_column(df,col)):
            df[col]= split_values_in_column(df,col)
            # Replace NAs with empty lists 
            mask_null= df[col].isnull()
            df[col].loc[mask_null]=df[col].loc[mask_null].apply(lambda x: [])
    return df


## Preprocessing Years related columns

In [6]:
raw_df['YearsCode']= clean_date(raw_df,'YearsCode')
raw_df['YearsCodePro']= clean_date(raw_df,'YearsCodePro')


## Preprocessing target column

In [7]:
df= clean_target(raw_df,'DevType')

In [8]:
df.shape

(71366, 19)

In [9]:
df['DevType'].unique()

array(['Senior Executive (C-Suite, VP, etc.)', 'Developer, back-end',
       'Developer, front-end', 'Developer, full-stack',
       'System administrator',
       'Developer, desktop or enterprise applications',
       'Developer, QA or test',
       'Data scientist or machine learning specialist',
       'Data or business analyst', 'Security professional', 'Educator',
       'Research & Development role', 'Developer, mobile',
       'Database administrator',
       'Developer, embedded applications or devices', 'Engineer, data',
       'Hardware Engineer', 'Product manager', 'Academic researcher',
       'Developer, game or graphics', 'Cloud infrastructure engineer',
       'Engineering manager', 'Developer Experience', 'Project manager',
       'DevOps specialist', 'Engineer, site reliability', 'Blockchain',
       'Developer Advocate', 'Scientist'], dtype=object)

## Preprocessing skills and languages related columns

In [10]:
df=split_multiple_answers(df.copy())

/tmp/ipykernel_67867/3509846276.py:87: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df[col].loc[mask_null]=df[col].loc[mask_null].apply(lambda x: [])
/tmp/ipykernel_67867/3509846276.py:87: SettingWithCopyWarning: 
A value is trying to be se

## Visually Verifying Preprocessed DataFrame

In [11]:
df.sample(5)


,DevType,EdLevel,YearsCode,YearsCodePro,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,OpSysProfessional use,ProfessionalTech,NEWCollabToolsHaveWorkedWith
72556,"Developer, front-end","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",5.0,0.0,"[HTML/CSS, JavaScript]","[Go, HTML/CSS, Java, JavaScript, Python]",[BigQuery],[],"[Amazon Web Services (AWS), Firebase]",[Amazon Web Services (AWS)],"[Next.js, Node.js, Nuxt.js, React, Vue.js]","[Next.js, React]",[],[],"[Docker, Make, npm, Yarn]","[Docker, Kubernetes, Terraform, Yarn]",[MacOS],"[DevOps function, Microservices, Automated tes...","[Jupyter Notebook/JupyterLab, Vim, Visual Stud..."
22706,"Developer, back-end","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",7.0,5.0,"[Java, Kotlin]","[Java, Kotlin]",[PostgreSQL],[PostgreSQL],[Amazon Web Services (AWS)],[Amazon Web Services (AWS)],[Angular],[Angular],[Apache Kafka],[Apache Kafka],[Maven (build tool)],[Maven (build tool)],[Ubuntu],[],[IntelliJ IDEA]
4031,"Developer, back-end","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",13.0,10.0,"[Bash/Shell (all shells), Groovy, Java, JavaSc...","[Bash/Shell (all shells), Go, Java, JavaScript...","[Cassandra, Couchbase, Elasticsearch, MySQL]","[Cassandra, Couchbase, Elasticsearch, MySQL]",[Amazon Web Services (AWS)],[Amazon Web Services (AWS)],[],[],[],[],"[Docker, Gradle, Homebrew, Kubernetes, Maven (...","[Docker, Gradle, Homebrew, Kubernetes, Maven (...",[MacOS],"[DevOps function, Microservices, Automated tes...","[Atom, IntelliJ IDEA, Sublime Text]"
22448,"Developer, front-end","Associate degree (A.A., A.S., etc.)",9.0,9.0,"[HTML/CSS, JavaScript]","[HTML/CSS, JavaScript]",[],[],[],[],"[Node.js, Vue.js]","[Node.js, Vue.js]",[],[],"[npm, pnpm, Vite, Webpack]","[npm, Vite]",[Windows],[],"[Sublime Text, VSCodium]"
74594,"Developer, front-end","Associate degree (A.A., A.S., etc.)",15.0,9.0,"[Bash/Shell (all shells), C, HTML/CSS, JavaScr...","[Assembly, Bash/Shell (all shells), C]","[Elasticsearch, MariaDB, MySQL, SQLite]","[MariaDB, MySQL, SQLite]","[Amazon Web Services (AWS), Vercel]","[Amazon Web Services (AWS), Vercel]","[Express, Flask, Next.js, Node.js, Nuxt.js, Re...",[],"[OpenGL, Scikit-Learn]",[],"[CMake, Docker, GNU GCC, Make, npm, Webpack, Y...","[CMake, GNU GCC, Make]","[Debian, MacOS, Other Linux-based]",[],"[Jupyter Notebook/JupyterLab, Neovim, Vim, Vis..."


## Exporting Preprocessed DataFrame in Pickle format

In [12]:

df.to_pickle("../data/processed/preprocessed_df.pkl")